## Using functools `@total_ordering` decorator

In [1]:
from functools import total_ordering

class Room:
    def __init__(self, name, length, width) -> None:
        self.name = name
        self.legth = length
        self.width = width
        self.square_feet = self.legth * self.width

@total_ordering
class House:
    def __init__(self, name, style) -> None:
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(room.square_feet for room in self.rooms)

    def add_room(self, room) -> None:
        return self.rooms.append(room)

    def __str__(self) -> str:
        return f"{self.name}: {self.living_space_footage} square foot {self.style}"

    def __eq__(self, other) -> bool:
        return self.living_space_footage == other.living_space_footage

    def __lt__(self, other) -> bool:
        return self.living_space_footage < other.living_space_footage


In [2]:
# Build a few houses and add roooms
h1 = House('h1', 'Cape')
r1 = Room('Master Bedroom', 14, 21)
r2 = Room('Living Room', 18, 20)
r3 = Room('Kitchen', 12, 16)
r4 = Room('Office',  12, 12)
rooms = [r1, r2, r3, r4]
for room in rooms:
    h1.add_room(room)

print(h1)

h1: 990 square foot Cape


In [3]:
h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))

print(h2)

h2: 846 square foot Ranch


In [4]:
h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))

print(h3)

h3: 1101 square foot Split


In [5]:
h1 < h2

False

In [6]:
h3 >= h2

True

In [7]:
print(min(h1, h2, h3))

h2: 846 square foot Ranch


In [8]:
print(max(h1, h2, h3))

h3: 1101 square foot Split


## Under the hood

Once you define `__eq__ (equal to)` and `__lt__ (less than)` methods, total_ordering takes care of the rest.

In [9]:
class House:
  def __eq__(self, other):
    pass  # You define this method

  def __lt__(self, other):
    pass  # You define this method

  # Methods created automatically by @total_ordering
  __le__ = lambda self, other: self < other or self == other  # Based on your __lt__ and __eq__
  __gt__ = lambda self, other: not (self < other or self == other)  # Logic derived from others
  __ge__ = lambda self, other: not (self < other)  # Derived logic
  __ne__ = lambda self, other: not self == other  # Based on your __eq__


In [10]:
help(total_ordering)

Help on function total_ordering in module functools:

total_ordering(cls)
    Class decorator that fills in missing ordering methods



This function is a class decorator that adds missing ordering methods to a class. 

It works by defining conversion rules for each comparison method (`__lt__`, `__le__`, `__gt__`, `__ge__`) based on the presence of other methods. 

If one comparison method is defined, the decorator fills in the missing ones based on the defined method(s).

For example, if a class defines `__lt__` and `__eq__`, the decorator will add `__le__`, `__gt__`, and `__ge__` based on the rules of comparison (`__le__` is the negation of `__gt__`, etc.).

The decorator ensures that a class has a consistent set of ordering methods, which can be convenient when working with objects that need to be compared and ordered.

In [11]:
def total_ordering(cls):
    """Class decorator that fills in missing ordering methods."""
    convert = {
        '__lt__': [('__gt__', lambda self, other: not (self < other or self == other)),
                    ('__le__', lambda self, other: self < other or self == other),
                    ('__ge__', lambda self, other: not self < other)],
        '__le__': [('__ge__', lambda self, other: not self < other),
                    ('__lt__', lambda self, other: self < other or self == other),
                    ('__gt__', lambda self, other: not (self < other or self == other))],
        '__gt__': [('__lt__', lambda self, other: not (self < other or self == other)),
                    ('__ge__', lambda self, other: not self < other),
                    ('__le__', lambda self, other: self < other or self == other)],
        '__ge__': [('__le__', lambda self, other: self < other or self == other),
                    ('__gt__', lambda self, other: not (self < other or self == other)),
                    ('__lt__', lambda self, other: not self < other)],
    }
    roots = set(dir(cls)) & {'__lt__', '__le__', '__gt__', '__ge__'}
    if not roots:
        raise ValueError(f"{cls.__name__} must define at least one ordering method: __lt__, __le__, __gt__, or __ge__")

    root = max(roots)  # Python 3.0-3.3 compatibility
    for opname, opfunc in convert[root]:
        if opname not in roots:
            opfunc.__name__ = opname
            opfunc.__qualname__ = f'{cls.__qualname__}.{opname}'
            setattr(cls, opname, opfunc)
    return cls
